In [ ]:
# For Users to Input their own file paths to their preprocessed data

# Path to the .csv file
FILE_PATH = 
#Example: "/Users/jacobellen/dropbox/Preprocessed_SHARE.csv"

# Preferred location/folder to save results
OUTPUT_FOLDER_NAME = 
#Example: "/Users/jacobellen/dropbox/"

In [ ]:
# Importing Packages
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import random

In [ ]:
# Loading in the data, imputing and formatting the outcome frailty variable

# Read in the dataset
data = pd.read_csv(FILE_PATH)

# Initial Data Information
data['Frailty_Score'].describe()

# Impute missing data by medican imputation
median_imp = list(data.loc[:, "pmh_heart_attack" : "drugs_for_other"].columns)
median_imp.extend(["ever_smoked_daily", "depression", "outpatient_surgery",
                   "vitals_measured_weight", "vitals_height", "vitals_BMI", "day_per_week_alcohol", 
                   "hospital_stays_lastyear"
                  ])
for col in median_imp:
    data[col].fillna(data[col].median(), inplace=True)
       
# Target Variable
data["Frailty_Score"] = data["Frailty_Score"].apply(lambda x: 1 if x == "Frail" else 0)

In [ ]:
# Initialize dictionaries for storing results

# Binary Classification Results
results = {
    "All": {"Accuracy": [], "AUC": [], "Sensitivity": [], "Specificity": [], "PPV": [], "NPV": []},
    "Normal": {"Accuracy": [], "AUC": [], "Sensitivity": [], "Specificity": [], "PPV": [], "NPV": []},
    "Prefrail": {"Accuracy": [], "AUC": [], "Sensitivity": [], "Specificity": [], "PPV": [], "NPV": []},
    "12_26_Months": {"Accuracy": [], "AUC": [], "Sensitivity": [], "Specificity": [], "PPV": [], "NPV": []},
    "27_33_Months": {"Accuracy": [], "AUC": [], "Sensitivity": [], "Specificity": [], "PPV": [], "NPV": []},
    "34_48_Months": {"Accuracy": [], "AUC": [], "Sensitivity": [], "Specificity": [], "PPV": [], "NPV": []},
}

# Triclass Classification Results
metrics_history = {
    'Accuracy': [],
    'F1': [],
    'AUC': [],
    'Sensitivity': [],
    'Specificity': [],
    'PPV': [],
    'NPV': []
}

# Initialize dictionaries to store all FPR, TPR, and AUC for different subsets across all runs
roc_data = {
    'All_Data': {'fpr': [], 'tpr': [], 'auc': []},
    '12-26 Months': {'fpr': [], 'tpr': [], 'auc': []},
    '27-33 Months': {'fpr': [], 'tpr': [], 'auc': []},
    '34-48 Months': {'fpr': [], 'tpr': [], 'auc': []},
}

# Variable importance lists
PredChange = []
AUCChange_List = [] 

# Random states
random_states = [1, 2, 3, 4, 5]

In [ ]:
# Setting random seed and train/test splits for reproducibility

SEED = 30
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Define StratifiedKFold for consistent splits
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Create Train-Test Splits for Binary Classification
binary_splits = list(skf.split(data.drop(columns=["Frailty_Score"]), data["Frailty_Score"]))

### Binary Classification Task

In [ ]:
# Looking at all the columns in the dataframe
data.columns

In [ ]:
# Looking at how the dataset is structured
data.head

In [ ]:
# Looking at distribution of the outcome frailty variable 
data.groupby(["Frailty_Score"]).count()

In [ ]:
# Functions needed to perform binary classification task

# Summarizing results
def summarize_and_save_results(results, folder_path):
    """
    Summarize and save model performance metrics for binary classification.

    Parameters:
    results (dict): Dictionary containing performance metrics for different subsets.
    folder_path (str): Path to the folder where results should be saved.

    Returns:
    None
    """
    summary = {}
    for subset, metrics in results.items():
        summary[subset] = {
            'Accuracy': f"{np.mean(metrics['Accuracy']):.4f} ± {np.std(metrics['Accuracy']):.4f}",
            'AUC': f"{np.mean(metrics['AUC']):.4f} ± {np.std(metrics['AUC']):.4f}",
            'Sensitivity': f"{np.mean(metrics['Sensitivity']):.4f} ± {np.std(metrics['Sensitivity']):.4f}",
            'Specificity': f"{np.mean(metrics['Specificity']):.4f} ± {np.std(metrics['Specificity']):.4f}",
            'PPV': f"{np.mean(metrics['PPV']):.4f} ± {np.std(metrics['PPV']):.4f}",
            'NPV': f"{np.mean(metrics['NPV']):.4f} ± {np.std(metrics['NPV']):.4f}"
        }

    # Save results to a DataFrame
    summary_df = pd.DataFrame.from_dict(summary, orient='index')
    output_file = os.path.join(folder_path, "Summary_Metrics.csv")
    summary_df.to_csv(output_file)
    print(summary_df)
    print(f"Summary metrics saved to: {output_file}")

# ROC Curve Analysis
def roc_analysis(y_test, y_prob):
    """
    Perform ROC analysis and plot the ROC curve for binary classification.

    Parameters:
    y_test (numpy.ndarray): True labels for the test set.
    y_prob (numpy.ndarray): Predicted probabilities for the test set.

    Returns:
    float: Optimal threshold for maximizing sensitivity and specificity.
    """
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    roc_auc = auc(fpr, tpr)

    # Optimal Threshold (Maximizes Sensitivity + Specificity)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # Plot ROC Curve
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {roc_auc:.2f}')
    plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', label=f'Optimal Threshold = {optimal_threshold:.2f}')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.show()

    print(f'Optimal Threshold for Sensitivity & Specificity: {optimal_threshold:.2f}')
    return optimal_threshold

# Define function to store ROC data for each subset
def store_roc_data(y_true, y_prob, key):
    """
    Store ROC curve data (FPR, TPR, and AUC) for a specific subset of data.

    Parameters:
    y_true (numpy.ndarray): True labels.
    y_prob (numpy.ndarray): Predicted probabilities.
    key (str): Subset identifier for storing data.

    Returns:
    None
    """
    # Ensure the key exists in the roc_data dictionary
    if key not in roc_data:
        roc_data[key] = {'fpr': [], 'tpr': [], 'auc': []}

    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    roc_data[key]['fpr'].append(fpr)
    roc_data[key]['tpr'].append(tpr)
    roc_data[key]['auc'].append(roc_auc)

# Helper function to calculate metrics and store them
def process_subset(model, X, y):
    """
    Evaluate a trained model on a test subset and calculate performance metrics.

    Parameters:
    model (keras.Model): Trained Keras model.
    X (numpy.ndarray): Features for the test set.
    y (numpy.ndarray): True labels for the test set.

    Returns:
    tuple: Metrics including accuracy, AUC, sensitivity, specificity, PPV, and NPV.
    """
    y_prob = model.predict(X).flatten()
    optimal_idx = np.argmax(roc_curve(y, y_prob)[1] - roc_curve(y, y_prob)[0])
    optimal_threshold = roc_curve(y, y_prob)[2][optimal_idx]
    y_pred = (y_prob >= optimal_threshold).astype(int)
    cm = confusion_matrix(y, y_pred)
    accuracy, sensitivity, specificity, ppv, npv = calculate_conf_metrics(cm)
    auc_score = roc_auc_score(y, y_prob)
    return accuracy, auc_score, sensitivity, specificity, ppv, npv

def calculate_conf_metrics(cm):
    """
    Calculate metrics (accuracy, sensitivity, specificity, PPV, and NPV) from a confusion matrix.

    Parameters:
    cm (numpy.ndarray): Confusion matrix (2x2 array).

    Returns:
    tuple: Metrics including accuracy, sensitivity, specificity, PPV, and NPV.
    """
    TN, FP, FN, TP = cm.ravel()
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) != 0 else 0
    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    ppv = TP / (TP + FP) if (TP + FP) != 0 else 0
    npv = TN / (TN + FN) if (TN + FN) != 0 else 0
    return accuracy, sensitivity, specificity, ppv, npv


In [ ]:
# Functions for Modeling for Binary Classification - 5 train/test splits that are averaged

for fold, (train_idx, test_idx) in enumerate(binary_splits):
    print(f"Binary Classification - Fold {fold+1}")
    
    # Split Data Using Predefined Indices
    train = data.iloc[train_idx]
    test = data.iloc[test_idx]
    
    # SMOTE Oversampling
    print("Train Class Distribution Before SMOTE:", Counter(train["Frailty_Score"]))
    X = train.drop(columns=["Frailty_Score", "Baseline_Frailty"])
    y = train["Frailty_Score"]
    smote = SMOTE(sampling_strategy=0.90, random_state=SEED)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    print("Train Class Distribution After SMOTE:", Counter(y_resampled))
    
    # Combine Resampled Data
    train = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), 
                       pd.DataFrame(y_resampled, columns=["Frailty_Score"])], axis=1)
    features = list(train.loc[:, train.columns != "Frailty_Score"].columns)
    output = "Frailty_Score"
    
    # Finalizing the Training Set
    train = pd.DataFrame(train)
    
    # Splitting Testing Set by Baseline Frailty
    test_normal = test[test['Baseline_Frailty'] == 0]
    test_prefrail = test[test['Baseline_Frailty'] == 1]
    
    # Dropping Baseline Frailty so it is not used as a predictor
    test_normal = test_normal.drop(columns=['Baseline_Frailty'], axis=1)
    test_prefrail = test_prefrail.drop(columns=['Baseline_Frailty'], axis=1)
    test = test.drop(columns=['Baseline_Frailty'], axis=1)

    # Finalizing Testing Sets
    test = pd.DataFrame(test)
    test_prefrail = pd.DataFrame(test_prefrail)
    test_normal = pd.DataFrame(test_normal)

    # Splitting Testing Set by Months Between Appointments
    test_12_26 = test[(test['Months_Between_Appointments'] >= 12) & (test['Months_Between_Appointments'] <= 26)]
    test_27_33 = test[(test['Months_Between_Appointments'] > 26) & (test['Months_Between_Appointments'] < 34)]
    test_34_48 = test[(test['Months_Between_Appointments'] >= 34) & (test['Months_Between_Appointments'] <= 48)]
    
    # Getting Size of Each Division by Time
    print("12-26 Group Size:", len(test_12_26))
    print("27-33 Group Size:", len(test_27_33))
    print("34-48 Group Size:", len(test_34_48))
    
    # Split train/test into features and labels
    X_train = train.drop('Frailty_Score', axis=1) 
    y_train = train.Frailty_Score  
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=SEED) 

    # Finalizing all the Test Subsets ("TestN" = Normal Patients; "TestF" = Prefrail Patients)
    testN = test_normal
    testF = test_prefrail
    X_test = test.drop('Frailty_Score', axis=1) 
    y_test = test.Frailty_Score

    # Initiating the Model
    model = Sequential()
    model.add(Dense(10, input_dim=(X_train.shape[1]), activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=0.001)
    
    # Compiling the Model
    model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=[
        keras.metrics.SensitivityAtSpecificity(0.5, num_thresholds=10),
        keras.metrics.SpecificityAtSensitivity(0.5, num_thresholds=10),
        keras.metrics.AUC(name='auc')]
    )
    
    # Training the Model
    history = model.fit(
    X_train, y_train, validation_data=(X_val, y_val),
    epochs=25, batch_size=64, verbose=1)

    # Getting Values for Feature Importance Analysis
    feature_names = X_train.columns.tolist()
    X_test = X_test.values

    # Optimal sensitivity/specifity for all patients using ROC
    y_prob = model.predict(X_test).flatten()
    optimal_threshold_all = roc_analysis(y_test, y_prob)
    y_pred = (y_prob >= optimal_threshold_all).astype(int)
    
    # Calculate importance scores for each feature 
    original_predictions = model.predict(X_test)
    y_pred_baseline = (original_predictions >= optimal_threshold_all).astype(int)
    importance_change = []
    auc_importance = []

    for feature_index in range(X_test.shape[1]):
        perturbed_data = X_test.copy()
        perturbed_data[:, feature_index] = np.random.permutation(perturbed_data[:, feature_index])
        perturbed_predictions = model.predict(perturbed_data)
        
        # Change in predictions
        importance_predchange = np.mean(np.abs(perturbed_predictions - original_predictions))
        importance_change.append(importance_predchange)
               
        # Calculate the change in AUC when the feature is perturbed
        baseline_auc = roc_auc_score(y_test, y_prob) 
        perturbed_prob = model.predict(perturbed_data).flatten()
        perturbed_auc = roc_auc_score(y_test, perturbed_prob)
        auc_drop = baseline_auc - perturbed_auc
        auc_importance.append(auc_drop)

    # Create a DataFrame containing the feature names and their importance scores
    importance_change = np.array(importance_change)
    Most_Important_Change = pd.DataFrame({
        'Variable': feature_names,
        'Importance': importance_change })
    importance_auc = np.array(auc_importance) 
    Most_Important_AUC = pd.DataFrame({
        'Variable': feature_names,
        'Importance': importance_auc })
    
    # Display the DataFrame
    print(Most_Important_Change)

    # Evaluation and Metrics
    metrics_lists = {
        'sensitivity': [], 'specificity': [], 'accuracy': [], 'f1': [],
        'precision': [], 'misclassification': [], 'npv': []
    }
    threshold_list = []

    # Evaluate on test sets
    for subset_name, subset_data in [
        ("All", test),
        ("Normal", testN),
        ("Prefrail", testF),
        ("12_26_Months", test_12_26),
        ("27_33_Months", test_27_33),
        ("34_48_Months", test_34_48),
    ]:
        X_test = subset_data.drop(columns=["Frailty_Score"])
        y_test = subset_data["Frailty_Score"]
        accuracy, auc_score, sensitivity, specificity, ppv, npv = process_subset(model, X_test, y_test)
        results[subset_name]['Accuracy'].append(accuracy)
        results[subset_name]['AUC'].append(auc_score)
        results[subset_name]['Sensitivity'].append(sensitivity)
        results[subset_name]['Specificity'].append(specificity)
        results[subset_name]['PPV'].append(ppv)
        results[subset_name]['NPV'].append(npv)
        y_prob_roc = model.predict(X_test).flatten()
        store_roc_data(y_test, y_prob_roc, subset_name)
    
    # Important variables
    PredChange.append(Most_Important_Change)
    AUCChange_List.append(Most_Important_AUC)


In [ ]:
# Follow-up Length AUC Figure for Binary Classification

# Define the groups to include in the plot
groups_to_plot = ["All", "12_26_Months", "27_33_Months", "34_48_Months"]

# After running 5-fold loop, compute the mean TPR at each FPR grid point
mean_fpr = np.linspace(0, 1, 100)

# Start plotting
fig, ax = plt.subplots(figsize=(10, 8))

# Define a consistent, publication-friendly color palette
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Update group labels
label_mapping = {
    "All": "All Patients",
    "12_26_Months": "12-26 Months",
    "27_33_Months": "27-33 Months",
    "34_48_Months": "34-48 Months"
}

# Iterate over the specified groups only
for i, group in enumerate(groups_to_plot):
    if group not in roc_data:
        continue
    
    # Get the data for the group
    data = roc_data[group]
    display_key = label_mapping.get(group, group)
    
    mean_tpr = np.zeros_like(mean_fpr)
    
    # Interpolate each ROC curve and add it to the mean_tpr
    for fpr, tpr in zip(data['fpr'], data['tpr']):
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
    
    mean_tpr /= len(data['fpr'])  # Average it
    mean_auc = auc(mean_fpr, mean_tpr)
    
    # Plot with improved formatting
    ax.plot(
        mean_fpr,
        mean_tpr,
        label=f'{display_key} (AUC = {mean_auc:.2f})',
        lw=2,
        color=colors[i % len(colors)]
    )

# Plot diagonal line for reference
ax.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=1)

# Improve axis titles and labels
ax.set_xlabel('False Positive Rate (1-Specificity)', fontsize=14)
ax.set_ylabel('True Positive Rate (Sensitivity)', fontsize=14)

# Improve the legend
ax.legend(
    loc='lower right',
    fontsize=12,
    frameon=True,
    edgecolor='black'
)

# Add gridlines for clarity
ax.grid(True, linestyle='--', alpha=0.5)

# Optimize spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Most Important Variables Code

# Concatenating dataframes into a single dataframe
combined_df_change = pd.concat(PredChange, ignore_index=True)

# Grouping by 'Variable' and computing mean and standard deviation of 'Importance'
result_df_change = combined_df_change.groupby('Variable').agg(
    Average_Importance=('Importance', 'mean'),
    Std_Dev_Importance=('Importance', 'std')
).reset_index()

# Sorting the resulting DataFrame by 'Average_Importance' in descending order
Change_DF = result_df_change.sort_values(by='Average_Importance', ascending=False).reset_index(drop=True)

print(Change_DF)


# Accuracy Most Important Variables

# Concatenating dataframes into a single dataframe
combined_df_auc = pd.concat(AUCChange_List, ignore_index=True)

# Grouping by 'Variable' and computing mean and standard deviation of 'Importance'
result_df_auc = combined_df_auc.groupby('Variable').agg(
    Average_AUC_Decrease=('Importance', 'mean'),
    Std_Dev_AUC=('Importance', 'std')
).reset_index()

# Sorting the resulting DataFrame by 'Average_Importance' in descending order
AUC_DF = result_df_auc.sort_values(by='Average_AUC_Decrease', ascending=False).reset_index(drop=True)

print(AUC_DF)

In [ ]:
# Summarize and save the results
summarize_and_save_results(results, OUTPUT_FOLDER_NAME)